<a href="https://colab.research.google.com/github/guarnieri0/GAIN/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import librerie**

In [24]:
import pandas as pd 

import json
import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy.linalg import norm
from numpy import savetxt
from sklearn import preprocessing

**Reading Dataset**

In [25]:
df = pd.read_json("dataset.json")
#df_past = pd.read_json("past_dataset.json")

**Deleting useless columns**

In [26]:
df = df.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)
#df_past = df_past.drop(['idx','testArgs','lastFailJob','engine','assignee','lastPassJob','projectIdx','validationIdx','templateIdx','triggerIdx','firstFailJob','lastFailJob','silent','status','reportLink'], axis = 1)

In [ ]:
#print(df_past)
print(df)

**Retrieving the Error Code and Error description**

In [ ]:
df_splitted= df['compareInfo'].str.split(pat=',', expand=True)
df_splitted = df_splitted.drop([0,1], axis = 1)
error_code = df_splitted.iloc[:,0].to_list()
error_description = df_splitted.iloc[:,1].to_list()
df.insert(5, 'error_description', error_description)
df.insert(6,'error_code',error_code)

"""df_splitted_past= df_past['compareInfo'].str.split(pat=',', expand=True)
df_splitted_past = df_splitted_past.drop([0,1], axis = 1)
error_code = df_splitted_past.iloc[:,0].to_list()
error_description = df_splitted_past.iloc[:,1].to_list()
df_past.insert(5, 'error_description', error_description)
df_past.insert(6,'error_code',error_code)"""

**Retrieving the test category**

In [ ]:
testname_splitted = df['testName'].str.split(pat='\\', expand = True)
testname_splitted = testname_splitted.drop([0,1,2,4,5], axis = 1)
print(testname_splitted)
testname = testname_splitted.iloc[:,0].to_list()
df.insert(7,'cat_name', testname)
print(df)

"""testname_past_splitted = df_past['testName'].str.split(pat='\\', expand = True)
testname_past_splitted = testname_past_splitted.drop([0,1,2,4,5], axis = 1)
print(testname_past_splitted)
testname = testname_past_splitted.iloc[:,0].to_list()
df_past.insert(7,'cat_name', testname)
print(df_past)"""


**Deleting redundant columns**

In [30]:
df = df.drop(['compareInfo','reportTitle'], axis = 1)
#df_past = df_past.drop(['compareInfo','reportTitle'], axis = 1)

In [31]:
print(df.columns)


Index(['testName', 'reportId', 'timestamp', 'error_description', 'error_code',
       'cat_name'],
      dtype='object')


**Filling the NULL value**

---

In [32]:
df['error_description'] = df['error_description'].fillna('None')
df['error_code'] = df['error_code'].fillna('None')
#df['error_description'] = df['error_description'].fillna('None')


**Encoding text features**

In [33]:
encoder_df_error_code = LabelEncoder()
encoder_df_error_description = LabelEncoder()
encoder_df_cat_name = LabelEncoder()



df['error_code'] = encoder_df_error_code.fit_transform(df['error_code'])
df['error_description'] = encoder_df_error_description.fit_transform(df['error_description'])
df['cat_name'] = encoder_df_cat_name.fit_transform(df['cat_name'])

"""df_past['error_code'] = encoder_df_error_code.fit_transform(df_past['error_code'])
df_past['error_description'] = encoder_df_error_description.fit_transform(df_past['error_description'])
df_past['cat_name'] = encoder_df_cat_name.fit_transform(df_past['cat_name'])"""

"df_past['error_code'] = encoder_df_error_code.fit_transform(df_past['error_code'])\ndf_past['error_description'] = encoder_df_error_description.fit_transform(df_past['error_description'])\ndf_past['cat_name'] = encoder_df_cat_name.fit_transform(df_past['cat_name'])"

In [34]:
print(df.max())

testName               test_verification
reportId               MNDORADOVAL-31553
timestamp            2022-10-10 02:26:24
error_description                      8
error_code                            13
cat_name                              44
dtype: object


In [35]:
ts =  pd.Timestamp(year = 2022, month = 10, day = 9, hour =0 , second = 0)
df_past = df.loc[lambda df: df['timestamp'] < ts]
df = df.drop(df[df.timestamp < ts].index)

In [36]:
df['timestamp'] = df['timestamp'].dt.date
df_past['timestamp'] = df_past['timestamp'].dt.date


In [46]:
print(df)
print(df_past)

                                             testName           reportId  \
0   \TestSuite\Dorado\PPTRecovery\PPTRecoveryTest_...  MNDORADOVAL-31553   
1   \TestSuite\Dorado\PPTRecovery\PPTRecoveryTest_...  MNDORADOVAL-31552   
2   \TestSuite\Generic\VoltageDetect\BO_11_VDTonIn...  MNDORADOVAL-31543   
3   \TestSuite\Generic\VoltageDetect\BO_10_VDTonWr...  MNDORADOVAL-31542   
4   \TestSuite\Generic\VoltageDetect\BO_12_VDTonId...  MNDORADOVAL-31541   
..                                                ...                ...   
78  \TestSuite\Dorado\SSR_EHR\EHR_05_ProgramFail_N...  MNDORADOVAL-31467   
79  \TestSuite\Dorado\SSR_EHR\SSR_11_EraseFail_OnM...  MNDORADOVAL-31466   
80  \TestSuite\Dorado\SSR_EHR\XMHR_03_ProgramFail_...  MNDORADOVAL-31465   
81  \TestSuite\Dorado\SSR_EHR\EHR_03_EraseFail_Not...  MNDORADOVAL-31464   
82  \TestSuite\Dorado\ShareBuffer\ShareReadBuf\Sha...  MNDORADOVAL-31463   

     timestamp  error_description  error_code  cat_name  
0   2022-10-10               

**Calculate the similarity matrix**

In [ ]:
similarity = np.empty((df_past.shape[0],df.shape[0]))
for i in range (0,df_past.shape[0],1):
  for j in range (0, df.shape[0],1):
   
    past_vector = np.array([df_past.iloc[i][3],df_past.iloc[i][4],df_past.iloc[i][5]])
    today_vector = np.array([df.iloc[j][3],df.iloc[j][4],df.iloc[j][5]])
    print(today_vector)

    #normalized_past_arr = preprocessing.normalize(past_vector)
    #normalized_today_arr = preprocessing.normalize(today_vector)
    
    #print(normalized_past_arr)
    #print(normalized_today_arr)
    #cosin similarity
    similarity[i,j] = np.dot(past_vector,today_vector)/(norm(past_vector)*norm(today_vector))
  

In [40]:
print(similarity)

[[0.97031905 0.97031905 0.98734854 ... 0.99350443 0.99350443 0.9974587 ]
 [0.98070529 0.98070529 0.94982068 ... 0.99274494 0.99274494 0.95358267]
 [0.98237209 0.98237209 0.97486611 ... 0.99998297 0.99998297 0.98177985]
 ...
 [1.         1.         0.97856258 ... 0.98211119 0.98211119 0.95652379]
 [0.98070529 0.98070529 0.94982068 ... 0.99274494 0.99274494 0.95358267]
 [0.98303645 0.98303645 0.9624536  ... 0.99756257 0.99756257 0.96761727]]


In [45]:
print(similarity.shape)

(117, 83)


In [42]:
savetxt('data.csv', similarity, delimiter = ',')

In [54]:
#max_position=np.amax(similarity.iloc[],axis = 0)
#print(max_position)
res = np.where(similarity[:,3] == np.amax(similarity[:,3]))
print(res)


#for i in res:
#  print(df_past.iloc[i])
#print(df)

(array([ 7,  9, 13, 14, 15, 18, 19, 21, 22, 23, 24, 28, 30]),)


In [56]:
print(df.iloc[3][:])

print(df_past.iloc[28][:])

testName             \TestSuite\Generic\VoltageDetect\BO_10_VDTonWr...
reportId                                             MNDORADOVAL-31542
timestamp                                                   2022-10-09
error_description                                                    6
error_code                                                           7
cat_name                                                            39
Name: 3, dtype: object
testName             \TestSuite\Generic\VoltageDetect\VDT_10_WriteW...
reportId                                             MNDORADOVAL-31417
timestamp                                                   2022-10-05
error_description                                                    6
error_code                                                           7
cat_name                                                            39
Name: 111, dtype: object
